In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/材料1.csv")

In [2]:
from sklearn.preprocessing import MinMaxScaler
from matplotlib import font_manager
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis, entropy

my_font = font_manager.FontProperties(fname="/System/Library/Fonts/STHeiti Medium.ttc")

magnetic_flux_density = data.iloc[:, 4:].values  # 从第5列开始为磁通密度数据
# print(magnetic_flux_density)

features = {
    "均值": [],
    "标准差": [],
    "峰值": [],
    "谷值": [],
    '主频率': [],
    '频谱能量': [],
    '频谱熵': [],
    '带宽': [],
    '波形面积': [],
    '均方根': [],
    '偏度': [],
    '峰度': []
}

mean_flux = np.mean(magnetic_flux_density, axis=1)  # 均值
std_flux = np.std(magnetic_flux_density, axis=1)  # 标准差
max_flux = np.max(magnetic_flux_density, axis=1)  # 峰值
min_flux = np.min(magnetic_flux_density, axis=1)  # 谷值
sqrt_root = np.mean(magnetic_flux_density, axis=1)  # 均方根
wire_square = np.trapz(magnetic_flux_density, axis=1)  # 波形面积
diff = skew(magnetic_flux_density, axis=1)  # 偏度
kurtosis = kurtosis(magnetic_flux_density, axis=1)  # 峰度

n_samples, n_timesteps = magnetic_flux_density.shape

# print(n_samples, n_timesteps)

main_frequencies = np.zeros(n_samples)
spectral_energies = np.zeros(n_samples)
spectral_entropies = np.zeros(n_samples)
bandwidths = np.zeros(n_samples)

sampling_rate = 1000

for i in range(n_samples):
    # 对每个时间序列计算FFT
    fft_vals = fft(magnetic_flux_density[i])
    fft_vals = np.abs(fft_vals[:n_timesteps // 2])  # 只取前半部分的频谱

    # 计算频率轴
    freqs = np.fft.fftfreq(n_timesteps, d=1 / sampling_rate)[:n_timesteps // 2]

    # 主频率 (找到最大振幅对应的频率)
    main_frequencies[i] = freqs[np.argmax(fft_vals)]

    # 频谱能量 (频谱平方的总和)
    spectral_energies[i] = np.sum(fft_vals ** 2)

    # 归一化频谱能量
    fft_vals_norm = fft_vals / np.sum(fft_vals)

    # 频谱熵 (归一化频谱的熵)
    spectral_entropies[i] = entropy(fft_vals_norm)

    # 带宽 (频谱的方差)
    mean_freq = np.sum(freqs * fft_vals_norm)
    bandwidths[i] = np.sqrt(np.sum((freqs - mean_freq) ** 2 * fft_vals_norm))

# sampling_rate = 1000
# N = len(magnetic_flux_density)
# signal_fft = fft(magnetic_flux_density, axis=1)
# freqs = fftfreq(N, 1 / sampling_rate)
# fft_magnitude = np.abs(signal_fft[:N // 2])
# freqs_positive = freqs[:N // 2]
# 
# main_f = freqs_positive[np.argmax(fft_magnitude)]  # 主频率
# f_energy = np.sum(fft_magnitude ** 2)  # 频谱能量
# 
# prob_fft = fft_magnitude / np.sum(fft_magnitude)
# f_shang = -np.sum(prob_fft * np.log2(prob_fft + np.finfo(float).eps))  # 频谱熵
# 
# width = np.sqrt(np.sum((freqs_positive ** 2) * (fft_magnitude ** 2)) / np.sum(fft_magnitude ** 2))  # 带宽

# for i in range(2550, 2554):
#     signal = magnetic_flux_density[i]
#     features["均值"].append(np.mean(signal))  # 磁通密度均值
#     features["标准差"].append(np.std(signal))  # 标准差
#     features["峰值"].append(np.max(signal))  # 最大值
#     features["谷值"].append(np.min(signal))  # 最小值
# 
#     sampling_rate = 1000
# 
#     features["均方根"].append(np.sqrt(np.mean(signal ** 2)))  # 均方根
#     features["波形面积"].append(np.trapz(signal))  # 波形面积
#     # features["零交叉率"].append(np.where(np.diff(np.sign(signal)))[0].size)  # 零交叉率
#     features["偏度"].append(skew(signal))  # 偏度
#     features["峰度"].append(kurtosis(signal))  # 峰度
# 
#     # 傅里叶变换
#     N = len(signal)
#     signal_fft = fft(signal)
#     freqs = fftfreq(N, 1 / sampling_rate)
#     fft_magnitude = np.abs(signal_fft[:N // 2])  # 只考虑正频率部分
#     freqs_positive = freqs[:N // 2]
# 
#     # 频域特征
#     features["主频率"].append(freqs_positive[np.argmax(fft_magnitude)])  # 主频率
#     # features["谐波频率"].append(freqs_positive[fft_magnitude > np.mean(fft_magnitude)])  # 谐波频率
#     features["频谱能量"].append(np.sum(fft_magnitude ** 2))  # 频谱能量
# 
#     # 计算频谱熵
#     prob_fft = fft_magnitude / np.sum(fft_magnitude)
#     features["频谱熵"].append(-np.sum(prob_fft * np.log2(prob_fft + np.finfo(float).eps)))
# 
# 
#     # 带宽
#     features["带宽"].append(np.sqrt(np.sum((freqs_positive ** 2) * (fft_magnitude ** 2)) / np.sum(fft_magnitude ** 2)))
# 
#     # 峰值检测
#     peaks, _ = find_peaks(signal)
#     if len(peaks) > 1:
#         peak_distances = np.diff(peaks)
#         peak_distance_mean = np.mean(peak_distances)  # 平均峰值间距
#     else:
#         peak_distance_mean = np.nan  # 如果没有足够的峰值
# 
#     # 相位偏移
#     features["相位偏移"].append(np.angle(signal_fft[np.argmax(fft_magnitude)]) ) # 主频率的相位

# df = pd.DataFrame.from_dict(features, orient='index')
# df.to_excel("~/Desktop/特征向量（材料一梯形波).xlsx")

# 创建特征数据集
# X = np.column_stack(
#     (max_flux, wire_square, min_flux, main_frequencies, spectral_energies, spectral_entropies, bandwidths, mean_flux,
#      sqrt_root, std_flux, diff, kurtosis))

X = np.column_stack(
    (max_flux, min_flux, mean_flux, std_flux))

# 将励磁波形（正弦波、三角波等）转换为分类标签
y = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3}).infer_objects(copy=False)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = MinMaxScaler()

xtrain_s = scaler.fit_transform(X_train)

xtest_s = scaler.transform(X_test)

# 使用随机森林模型进行分类
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 预测测试集
y_pred = clf.predict(X_test)

# 输出分类报告
classification_report_output = classification_report(y_test, y_pred)
print(classification_report_output)

/var/folders/gy/6wf0tc3n7276h9vd9dqcpkvw0000gn/T/ipykernel_21894/3623283342.py:148: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3}).infer_objects(copy=False)


              precision    recall  f1-score   support

           1       1.00      1.00      1.00       344
           2       0.99      1.00      0.99       405
           3       1.00      0.98      0.99       271

    accuracy                           0.99      1020
   macro avg       1.00      0.99      0.99      1020
weighted avg       0.99      0.99      0.99      1020



In [4]:
""

''

In [5]:
df = pd.read_excel("data/附件二（测试集）.xlsx")
df

,序号,温度，oC,频率，Hz,磁芯材料,0（磁通密度B，T）,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1,25,56180,材料1,-0.101398,-0.099219,-0.097045,-0.094873,-0.092701,-0.090534,...,-0.106739,-0.106970,-0.107198,-0.107405,-0.107564,-0.107633,-0.107374,-0.106567,-0.105241,-0.103473
1,2,25,125890,材料1,-0.031775,-0.031081,-0.030391,-0.029704,-0.029019,-0.028333,...,-0.034050,-0.034100,-0.034136,-0.034158,-0.034159,-0.034103,-0.033926,-0.033588,-0.033087,-0.032461
2,3,25,224180,材料1,-0.066871,-0.065300,-0.063737,-0.062185,-0.060640,-0.059101,...,-0.074207,-0.074200,-0.074168,-0.074077,-0.073844,-0.073352,-0.072520,-0.071357,-0.069958,-0.068436
3,4,25,282480,材料1,-0.023190,-0.022634,-0.022084,-0.021540,-0.020998,-0.020455,...,-0.027047,-0.027007,-0.026898,-0.026682,-0.026344,-0.025900,-0.025387,-0.024845,-0.024297,-0.023745
4,5,50,199170,材料1,-0.077613,-0.075852,-0.074096,-0.072348,-0.070611,-0.068883,...,-0.084331,-0.084361,-0.084376,-0.084366,-0.084294,-0.084046,-0.083461,-0.082435,-0.081015,-0.079358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,25,89080,材料4,-0.000273,-0.000175,-0.000077,0.000021,0.000118,0.000216,...,-0.001246,-0.001149,-0.001052,-0.000955,-0.000858,-0.000760,-0.000663,-0.000565,-0.000468,-0.000370
76,77,25,251170,材料4,0.000046,0.000120,0.000194,0.000267,0.000341,0.000414,...,-0.000690,-0.000617,-0.000543,-0.000470,-0.000396,-0.000322,-0.000249,-0.000175,-0.000101,-0.000028
77,78,50,79470,材料4,0.000279,0.000624,0.000968,0.001311,0.001654,0.001998,...,-0.003156,-0.002813,-0.002471,-0.002127,-0.001784,-0.001440,-0.001097,-0.000754,-0.000410,-0.000065
78,79,50,251170,材料4,-0.000708,-0.000571,-0.000435,-0.000298,-0.000162,-0.000026,...,-0.002069,-0.001933,-0.001798,-0.001662,-0.001526,-0.001390,-0.001253,-0.001117,-0.000981,-0.000844


In [6]:
filtered_df = df.loc[df['磁芯材料'] == '材料1']
filtered_df

,序号,温度，oC,频率，Hz,磁芯材料,0（磁通密度B，T）,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1,25,56180,材料1,-0.101398,-0.099219,-0.097045,-0.094873,-0.092701,-0.090534,...,-0.106739,-0.106970,-0.107198,-0.107405,-0.107564,-0.107633,-0.107374,-0.106567,-0.105241,-0.103473
1,2,25,125890,材料1,-0.031775,-0.031081,-0.030391,-0.029704,-0.029019,-0.028333,...,-0.034050,-0.034100,-0.034136,-0.034158,-0.034159,-0.034103,-0.033926,-0.033588,-0.033087,-0.032461
2,3,25,224180,材料1,-0.066871,-0.065300,-0.063737,-0.062185,-0.060640,-0.059101,...,-0.074207,-0.074200,-0.074168,-0.074077,-0.073844,-0.073352,-0.072520,-0.071357,-0.069958,-0.068436
3,4,25,282480,材料1,-0.023190,-0.022634,-0.022084,-0.021540,-0.020998,-0.020455,...,-0.027047,-0.027007,-0.026898,-0.026682,-0.026344,-0.025900,-0.025387,-0.024845,-0.024297,-0.023745
4,5,50,199170,材料1,-0.077613,-0.075852,-0.074096,-0.072348,-0.070611,-0.068883,...,-0.084331,-0.084361,-0.084376,-0.084366,-0.084294,-0.084046,-0.083461,-0.082435,-0.081015,-0.079358
5,6,25,282480,材料1,-0.011507,-0.011253,-0.011001,-0.010750,-0.010497,-0.010244,...,-0.012991,-0.012892,-0.012794,-0.012694,-0.012592,-0.012481,-0.012349,-0.012187,-0.011987,-0.011756
6,7,50,56160,材料1,-0.035798,-0.035013,-0.034231,-0.033448,-0.032664,-0.031883,...,-0.040471,-0.040178,-0.039886,-0.039586,-0.039273,-0.038894,-0.038426,-0.037890,-0.037264,-0.036558
7,8,50,89210,材料1,-0.027493,-0.026872,-0.026253,-0.025635,-0.025018,-0.024401,...,-0.031639,-0.031403,-0.031158,-0.030887,-0.030565,-0.030183,-0.029744,-0.029251,-0.028706,-0.028113
8,9,70,158740,材料1,-0.033539,-0.032832,-0.032128,-0.031422,-0.030721,-0.030026,...,-0.037110,-0.036849,-0.036585,-0.036317,-0.036044,-0.035765,-0.035480,-0.035169,-0.034768,-0.034214
9,10,70,199180,材料1,-0.020561,-0.020113,-0.019667,-0.019221,-0.018774,-0.018332,...,-0.023029,-0.022861,-0.022691,-0.022520,-0.022347,-0.022170,-0.021976,-0.021732,-0.021408,-0.021005


In [7]:
test_flux_density = filtered_df.iloc[:, 4:].values  # 从第5列开始为磁通密度数据

test_mean_flux = np.mean(test_flux_density, axis=1)  # 均值
test_std_flux = np.std(test_flux_density, axis=1)  # 标准差
test_max_flux = np.max(test_flux_density, axis=1)  # 峰值
test_min_flux = np.min(test_flux_density, axis=1)  # 谷值

test_X = np.column_stack((test_max_flux, test_min_flux, test_mean_flux, test_std_flux))

In [8]:
test_y_pred = clf.predict(test_X)
print(test_y_pred)
print(type(test_y_pred))

# df = pd.DataFrame(test_y_pred)
# 
# df.to_excel("~/Desktop/1.xlsx", index=False)

[2 2 2 2 2 3 3 3 3 3 1 1 1 1 1 2 2 2 2 2]
<class 'numpy.ndarray'>


In [9]:
# import matplotlib.pyplot as plt
# 
# # lst = [0, 4, 14, 24, 34, 44, 54, 64, 74, 79]
# plt.figure(figsize=(10, 6))
# for i in range(5):
#     plt.plot(test_flux_density[i], label=f'Sample {i+1}', alpha=0.7) 
# 
# # 图形设置
# plt.title('Magnetic Flux Density Over Time (Sampled Points)')
# plt.xlabel('Sample Points')
# plt.ylabel('Magnetic Flux Density (T)')
# plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1), ncol=1, fontsize='small')
# plt.grid(True)
# plt.tight_layout()
# 
# # 显示图像
# plt.show()

In [10]:
# [2 2 2 2 2 3 3 3 3 3 1 1 1 1 1 2 2 2 2 2]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# 使用KNN算法
knn_clf = KNeighborsClassifier(n_neighbors=5)  # 选择5个邻居作为示例
knn_clf.fit(X_train, y_train)

# 预测测试集
knn_pred = knn_clf.predict(X_test)

# 输出分类报告
classification_report_output = classification_report(y_test, knn_pred)
print(classification_report_output)

In [ ]:
from xgboost import XGBClassifier

# 使用XGBoost分类器
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  # 设置参数
xgb_clf.fit(X_train, y_train)

# 预测测试集
xgb_predictions = xgb_clf.predict(X_test)

# 输出分类报告
classification_report_output = classification_report(y_test, xgb_predictions)
print("Classification Report:\n", classification_report_output)
